# RECOMMEND_SYSTEMS
### Võ Hửu Nghĩa

#### IMPORT CÁC THƯ VIỆN CẦN THIẾT

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

#### KẾT NỐI VỚI CƠ SỞ DỮ LIỆU SQL SERVER

In [2]:
server = 'VoHuuNghia'  # Change this to your SQL Server hostname
database = 'library_warehouse'  # Change this to your database name
driver = '{ODBC Driver 17 for SQL Server}' 

#### LẤY DỮ LIỆU TỪ BẢNG "DimBook" CHO VÀO DATAFRAME

In [3]:
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    cursor.execute("SELECT @@version;")
    row = cursor.fetchone()
    print("Connected to SQL Server. Server version:", row[0])
    getBookQuery = "select * from DimBook"
    df_Book = pd.read_sql(getBookQuery, conn)
except pyodbc.Error as e:
    print(f"Error connecting to SQL Server: {e}")

Connected to SQL Server. Server version: Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Express Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 22631: ) (Hypervisor)



C:\Users\vhngh\AppData\Local\Temp\ipykernel_10024\194929899.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Book = pd.read_sql(getBookQuery, conn)


In [4]:
df_Book.head()

,Tailieu_ID,Tacgia_ID,Tailieu_Name,MaTailieu
0,12,NaN,$aBài tập đại số tuyến tính và hình học giải t...,SK020000015
1,31,11.0,$aThêm những hiểu biết về Hồ Chí Minh/$cĐỗ Qua...,SK020000032
2,38,NaN,$aChủ tịch Hồ Chí Minh ở pháp năm 1946,SK020000040
3,44,22.0,$aBác Hồ Bác Tôn và Các Anh/$cBảo Định Giang,SK020000047
4,55,NaN,$aThơ văn Hồ Chí Minh:$bTác phẩm chọn lọc dùng...,SK020000058


#### Tiền xử lí dữ liệu (Chỉ giữ lại tên sách, loại bỏ các kí tự đặc biệt và viết thường tất cả)

In [5]:
def clean_name(tailieu_name):
    match = re.search(r'\$a([^$]*)', tailieu_name)
    if match:
        clean_name = match.group(1).strip()
        clean_name = re.sub(r'[/:]', '', clean_name).lower()
        return clean_name
    return tailieu_name

# # # import re

# # # def clean_name(tailieu_name):
# # #     # Tách phần tên trước dấu "$c"
# # #     parts = tailieu_name.split('$c')
# # #     name = parts[0].strip()
    
# # #     # Xử lý phần tên trước dấu ":"
# # #     colon_index = name.find(':')
# # #     if colon_index != -1:
# # #         name = name[:colon_index]
    
# # #     # Loại bỏ các ký tự không mong muốn và chuyển về chữ thường
# # #     name = re.sub(r'[/:]', '', name).lower()
    
# # #     return name

#### Gán ngược lại dataframe

In [6]:
df_Book['Tailieu_Name'] = df_Book['Tailieu_Name'].apply(clean_name)

In [67]:
df_Book.sample(20)

,Tailieu_ID,Tacgia_ID,Tailieu_Name,MaTailieu,vector
33047,36277,27824.0,nghiên cứu các tác động và đề ra giải pháp giả...,SK120036281,"[-0.07247912, 0.39655203, 0.4841868, -0.195594..."
29159,32405,19787.0,giáo trình kỹ thuật lập trình c,SK110032406,"[0.07958836, 0.978128, 1.7740355, -0.24572226,..."
63,417,182.0,bất ngờ lý thú trong vật lý,SK020000426,"[0.53416836, 0.26184127, 0.91971874, -0.717942..."
16492,15851,14679.0,oxford english for computing,SK060015855,"[0.17715442, -0.13083392, -0.9664641, 0.662651..."
12566,6302,6902.0,cơ học đất cho đất không bão hòa,SK030006304,"[0.11325182, 0.9679328, 1.0088016, -0.4365633,..."
19781,22140,19667.0,con đường giải mã văn học trung đại việt nam,SK070022141,"[0.086278245, 0.83599603, 0.5206843, -0.432416..."
3783,6968,6389.0,công nghệ chế tạo và ứng dụng của cặp nhiệt,SK030006972,"[-0.5530004, 0.8018716, 1.2654529, -0.36253503..."
13088,3626,2848.0,ngữ pháp tiếng việt,SK020003650,"[0.13882056, 0.63750774, 0.8557782, -0.6439130..."
92,472,238.0,công nghệ sinh học,SK020000481,"[-0.056030527, 0.8049945, 1.0823617, -0.767467..."
12666,16082,14848.0,thực đơn bổ dưỡng cho trẻ,SK060016083,"[0.4183617, 0.5396015, 0.49808264, -0.18575005..."


In [8]:
df_Book.tail()

,Tailieu_ID,Tacgia_ID,Tailieu_Name,MaTailieu
34995,38220,29276.0,hoàn thiện công tác đào tạo nguồn nhân lực tại...,SK130038222
34996,38216,29273.0,kế toán tiền lương và các khoản trích theo lươ...,SK130038218
34997,38192,29245.0,kế toán xác định kết quả hoạt động kinh doanh ...,SK130038197
34998,38194,29247.0,kế toán xác định kết quả kinh doanh và phân tí...,SK130000001
34999,38195,25987.0,quy trình kiểm toán doanh thu và nợ phải thu k...,SK130000002


#### Tạo ra một ma trận đặc trưng sử dụng kỹ thuật TF-IDF

In [9]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_Book['Tailieu_Name'])

#### Huấn lệnh mô hình bằng thuật toán KNN

In [10]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(tfidf_matrix)

NearestNeighbors(metric='cosine', n_neighbors=10)

#### Thực thi mô hình

In [11]:
def recommend_books(keyword):
    keyword_cleaned = re.sub(r'[/:]', '', keyword.lower())
    keyword_tfidf = vectorizer.transform([keyword_cleaned])
    distances, indices = knn.kneighbors(keyword_tfidf)
    
    recommendations = []
    for index in indices.flatten():
        recommendations.append(df_Book.iloc[index]['Tailieu_Name'])
    
    return recommendations

In [12]:

keyword = "lập trình python"
recommendations = recommend_books(keyword)

print(f"Gợi ý sách cho từ khóa '{keyword}':")
for i, book in enumerate(recommendations, 1):
    print(f"{i}. {book}")

Gợi ý sách cho từ khóa 'lập trình python':
1. lập trình c #
2. lập trình c #
3. lập trình c#
4. giáo trình lập trình c
5. giáo trình lập trình c - tập 2 
6. giáo trình lập trình c - tập 1 
7. lập trình bằng c++
8. giáo trình lập trình ứng dụng c++ lập trình lập trình hướng đối tượng lý thuyết và bài tập
9. kỹ thuật lập trình c
10. kỹ thuật lập trình c


In [13]:
keyword = "hồ chí minh"
recommendations = recommend_books(keyword)
print(f"Gợi ý sách cho từ khóa '{keyword}':")
for i, book in enumerate(recommendations, 1):
    print(f"{i}. {book}")

Gợi ý sách cho từ khóa 'hồ chí minh':
1. hồ chí minh 
2. báo chí hồ chí minh 
3. minh triết hồ chí minh
4. hồ chí minh toàn tập
5. hồ chí minh tuyển tập
6. thành phố hồ chí minh
7. thơ văn hồ chí minh
8. hồ chí minh tiểu sử
9. danh ngôn hồ chí minh 
10. danh ngôn hồ chí minh 


In [14]:
keyword = "vi phân"
recommendations = recommend_books(keyword)
print(f"Gợi ý sách cho từ khóa '{keyword}':")
for i, book in enumerate(recommendations, 1):
    print(f"{i}. {book}")

Gợi ý sách cho từ khóa 'vi phân':
1. hình học vi phân
2. bài tập phương trình vi phân 
3. vi xử lý - máy vi tính
4. phép tính vi phân và tích phân của hàm một biến
5. kỹ thuật vi xử lý và máy vi tính 
6. lý thuyết chuỗi và phương trình vi phân
7. giải tích 1 - phép tính vi phân và tích phân của hàm một biến
8. hàm nhiều biến phương trình vi phân
9. cơ sở công nghệ vi điện tử và vi hệ thống
10. vi xử lý 2


In [15]:
keyword = "Việt nam"
recommendations = recommend_books(keyword)
print(f"Gợi ý sách cho từ khóa '{keyword}':")
for i, book in enumerate(recommendations, 1):
    print(f"{i}. {book}")

Gợi ý sách cho từ khóa 'Việt nam':
1. việt nam
2. Công nghiệp việt nam 
3. công nghiệp việt nam
4. hướng nghiệp việt nam =
5. pháp luật việt nam
6. ngữ pháp việt nam
7. giáo dục việt nam
8. mỹ thuật việt nam
9. thuốc việt nam 
10. thi nhân việt nam


In [16]:
from gensim.models import Word2Vec

#### Tạo một danh sách các từ

In [17]:
sentences = df_Book['Tailieu_Name'].apply(lambda x: x.split()).tolist()

#### Huấn lệnh mô hình với thuật toán Word2Vec

In [18]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

#### Hàm xử lí đầu vào

In [19]:

def vectorize_sentence(sentence, model):
    # Bước 1: Chia câu thành các từ và loại bỏ các từ không có trong mô hình
    words = [word for word in sentence.split() if word in model.wv]
    
    # Bước 2: Lấy vector của mỗi từ từ mô hình Word2Vec
    word_vectors = [model.wv[word] for word in words]
    
    # Bước 3: Nếu không có từ nào được tìm thấy trong mô hình, trả về vector không
    if not word_vectors:
        return np.zeros(model.vector_size)
    
    # Bước 4: Tính trung bình các vector từ để tạo vector câu
    return np.mean(word_vectors, axis=0)

#### Thêm một cột "vector" chứa các vector đại diện cho các câu/đoạn văn tương ứng trong cột "Tailieu_Name"

In [20]:
df_Book['vector'] = df_Book['Tailieu_Name'].apply(lambda x: vectorize_sentence(x, model))

In [21]:
df_Book.head()

,Tailieu_ID,Tacgia_ID,Tailieu_Name,MaTailieu,vector
0,12,NaN,bài tập đại số tuyến tính và hình học giải tích,SK020000015,"[0.08997024, 0.7434506, 0.88975024, -0.5978980..."
1,31,11.0,thêm những hiểu biết về hồ chí minh,SK020000032,"[0.44575197, 0.45674953, 0.23920594, -0.600260..."
2,38,NaN,chủ tịch hồ chí minh ở pháp năm 1946,SK020000040,"[0.37616098, 0.31022537, 0.23085707, -0.458949..."
3,44,22.0,bác hồ bác tôn và các anh,SK020000047,"[0.13843182, 0.5231753, 0.31497815, -0.4346113..."
4,55,NaN,thơ văn hồ chí minh,SK020000058,"[0.3571691, 0.59721047, 0.1613898, -0.8461698,..."


In [22]:
X = np.vstack(df_Book['vector'].values)
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(X)

NearestNeighbors(metric='cosine')

In [23]:
def recommend_books(keyword):
    keyword_cleaned = re.sub(r'[/:]', '', keyword.lower())
    keyword_vector = vectorize_sentence(keyword_cleaned, model)
    distances, indices = knn.kneighbors([keyword_vector])
    
    recommendations = []
    for index in indices.flatten():
        recommendations.append(df_Book.iloc[index]['Tailieu_Name'])
    
    return recommendations

In [24]:
keyword = "data"
recommendations = recommend_books(keyword)

print(f"Gợi ý sách cho từ khóa '{keyword}':")
for i, book in enumerate(recommendations, 1):
    print(f"{i}. {book}")

Gợi ý sách cho từ khóa 'data':
1. data processing;
2. digital data communications
3. automatic data processing systems
4. accounting information systems
5. systems development


In [25]:
keyword = "thuật toán"
recommendations = recommend_books(keyword)

print(f"Gợi ý sách cho từ khóa '{keyword}':")
for i, book in enumerate(recommendations, 1):
    print(f"{i}. {book}")

Gợi ý sách cho từ khóa 'thuật toán':
1. thuật toán và độ phức tạp thuật toán
2. từ thuật toán đến chương trình
3. đồ thị và các thuật toán
4. bài tập tính toán kỹ thuật lạnh
5. giáo trình thuật toán và kỹ thuật lập trình pascal 


In [26]:
keyword = "dữ liệu"
recommendations = recommend_books(keyword)

print(f"Gợi ý sách cho từ khóa '{keyword}':")
for i, book in enumerate(recommendations, 1):
    print(f"{i}. {book}")

Gợi ý sách cho từ khóa 'dữ liệu':
1. nhiên liệu sạch
2. vật liệu composite 
3. vật liệu bôi trơn
4. vật liệu hàn
5. vật liệu dệt
